# Recurrent Neural Network

## Part 1 - Data Preprocessing

### Importing the libraries

In [92]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

### Importing the training set

In [96]:
dataset_train = pd.read_csv("total_data.csv")
training_set = dataset_train.iloc[:,1:6].values
print(training_set.shape)

(2710, 5)


### Feature Scaling

In [97]:
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range = (0,1))
training_set_scaled = sc.fit_transform(training_set)
print(training_set_scaled.shape)

(2710, 5)


### Creating a data structure with 60 timesteps and 1 output

In [98]:
X_train = []
y_train = []
time_step = 30
for i in range(time_step, len(training_set_scaled)):
  X_train.append(training_set_scaled[i-time_step : i, :])
  y_train.append(training_set_scaled[i,0])
X_train, y_train = np.array(X_train), np.array(y_train)
print(X_train.shape)

(2680, 30, 5)


### Reshaping

In [8]:
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 5))

## Part 2 - Building and Training the RNN

### Importing the Keras libraries and packages

In [10]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

### Initialising the RNN

In [11]:
regressor = Sequential()

### Adding the first LSTM layer and some Dropout regularisation

In [12]:
regressor.add(LSTM(units= 50, return_sequences=True, input_shape =(X_train.shape[1], 5) ))
regressor.add(Dropout(0.2))

### Adding a second LSTM layer and some Dropout regularisation

In [13]:
regressor.add(LSTM(units= 50, return_sequences=True))
regressor.add(Dropout(0.2))

### Adding a third LSTM layer and some Dropout regularisation

In [14]:
regressor.add(LSTM(units= 50, return_sequences=True))
regressor.add(Dropout(0.2))

### Adding a fourth LSTM layer and some Dropout regularisation

In [15]:
regressor.add(LSTM(units= 50, return_sequences=False))
regressor.add(Dropout(0.2))

### Adding the output layer

In [43]:
regressor.add(Dense(units=5))

### Compiling the RNN

In [44]:
regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')

### Fitting the RNN to the Training set

In [45]:
regressor.fit(X_train, y_train, epochs = 100, batch_size= 32)

Epoch 1/100
84/84 [==============================] - 5s 60ms/step - loss: 0.0775
Epoch 2/100
84/84 [==============================] - 5s 60ms/step - loss: 0.0395
Epoch 3/100
84/84 [==============================] - 5s 62ms/step - loss: 0.0219
Epoch 4/100
84/84 [==============================] - 5s 62ms/step - loss: 0.0123
Epoch 5/100
84/84 [==============================] - 5s 62ms/step - loss: 0.0073
Epoch 6/100
84/84 [==============================] - 5s 62ms/step - loss: 0.0051
Epoch 7/100
84/84 [==============================] - 5s 61ms/step - loss: 0.0038
Epoch 8/100
84/84 [==============================] - 5s 61ms/step - loss: 0.0030
Epoch 9/100
84/84 [==============================] - 5s 60ms/step - loss: 0.0023
Epoch 10/100
84/84 [==============================] - 5s 59ms/step - loss: 0.0019
Epoch 11/100
84/84 [==============================] - 5s 60ms/step - loss: 0.0013
Epoch 12/100
84/84 [==============================] - 5s 61ms/step - loss: 0.0011
Epoch 13/100
84/84 [=====

## Part 3 - Making the predictions 

In [99]:

real_stock_price = training_set[(len(training_set))-time_step:,0:5]
real_stock_price = np.array(real_stock_price)
print(real_stock_price.shape)

(30, 5)


In [101]:
dataset_total = training_set
inputs = dataset_total[len(dataset_total) - time_step:]
inputs = inputs.reshape(-1,5)
inputs = sc.transform(inputs)
print(inputs.shape)

(30, 5)


In [102]:
X_test = []
for i in range(time_step, time_step+1):
    X_test.append(inputs[i-time_step:i, :])
X_test = np.array(X_test)
print(X_test.shape)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 5))
print(X_test.shape)

(1, 30, 5)
(1, 30, 5)


In [103]:


predicted_stock_price = regressor.predict(X_test)
print(predicted_stock_price)
predicted_stock_price = np.array(predicted_stock_price)
print(predicted_stock_price.shape)
predicted_stock_price.reshape(1,5)


[[0.9742  0.9742  0.97411 0.9742  0.9742 ]]
(1, 5)


array([[0.9742 , 0.9742 , 0.97411, 0.9742 , 0.9742 ]], dtype=float32)

In [104]:
r = sc.inverse_transform(predicted_stock_price)
r = np.array(r)

print('    open','          high','       low','     close','     volume')
print(r)

    open           high        low      close      volume
[[1.29105e+04 1.29260e+04 1.27593e+04 1.28356e+04 1.76499e+09]]
